# Train Filter

This notebook is designed to train the filter for the model. The filter is currently designed to take sample made up of a time series with three channels (one from each of the LIGO/Virgo detectors). Each of these samples contains 2s of either just noise or a gravitational wave overlayed with noise. The data is sampled at 2048 Hz.The filter will then try to reconstruct the gravitational wave. Or in the case of just noise, the ground truth is just a constant line at 0.

The notebook is recommended for training on a GPU, only. This is because the LSTM cells incur significant CPU overhead, when not using the explicit CUDA implementation and a lot of overhead even then. And more significantly, the loss function needs to be calculated between two tensors of shape [4096,3], when training the model for a specific purpose (reproduce the gravitational-wave or reduce noise) other than detection, also producing very significant CPU overhead. This leads to the model taking about 30 hrs/epoch, when training on a TPU.


### Reason for the filter

The goal of the filter currently is to extract the gravitational wave burried in detector noise and reconstruct it.

The long-term goal for the filter is to produce a time series, where the same possible gravitational wave signal is still present, but with reduced noise. This is an extrapolation from [this paper](https://arxiv.org/pdf/1908.03151.pdf). This paper shows, that the model, that was the basis for the actitecture of my detection model is extemely sensitive to signal to noise ratio(SNR). A generalization of this fact to all CNNs can be defended by the fact that the results in the competition appear to have a very hard cap of an AUC of about 0.88. To go above this threshold requires significant additional computational performance by using many different models in an ensemble. This only slightly appears to improve the score and is true for 2D CNNs as well as 1D CNNs and in ensembles of 1D and 2D CNNs. This small increase leads me to 2 conclusions:

1. The different models with vastly different model architectures have the same samples, where they cannot make adequate predictions.
2. The small increase of accuracy gained from ensembling large quantities of models means that 1. is only not true for a small number of samples, likely on the fringes of their effective SNRs.

This leads me to my working theory, that since models appear to universally react very strongly to changes in the SNR, boosting the SNR is critical to improving model performance. Small improvements would likely lead to a huge gain in performance.

For this reason the filter is designed to act analogously to active noise cancellation in headphones. Not eliminating all noise, but reducing the noise level to increase SNR. This is currently not what the filter is doing. 

But this is the first version and making the filter do what it should necessitates making a SNR-based loss function. Which will be done in the future. This model should then increase the efficacy of all detection models without bottelnecking the efficacy to the worst model in the stack, as is currently the case.

All of this is, of course speculation and needs to be verified via tests.

Regardless, reducing the noise should be more beneficial to the detection network than reproducing a noise-free version of the wave. In the latter case, the combined model (filter as input to the detection model) will be bottlenecked by the worse network, since both the filter and the body of the detecion network have a very similar purpose. Fine-tuning can somewhat negate this issue and allow the combined model to perform better than the two individual models, but will lead to a very unwieldy model, which can easily overfit.

## Setup

### Imports

In [1]:
# General
import os
import numpy as np
import sys
from glob import glob
import random
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

#ML
import tensorflow as tf
from tensorflow.keras import mixed_precision, layers
mixed_precision.set_global_policy("mixed_float16")
AUTO = tf.data.experimental.AUTOTUNE

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2080 Ti, compute capability 7.5


In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Params

In [3]:
Params = {
    "batch_size": 128
}

## Convenience Functions

### load_dataset()

This function will load the dataset from different `tfrec` files. The datasets include a "ground truth", which is a time series containing either the un-whitened gravitational wave or a vector of 0. The ground truth tensors were not whitened, because the loss function used was the root mean squared error (RMSE), which lead to ill-posed models in cases where the signal was whitened, because in those cases there was only one time point with a high amplitude, compared to all others, that were very close to 0. RMSE was chosen, because in this case it was not necessary to reproduce the wavelet as exactly as possible, but to make a clear distinction between samples only containing noise and those with a signal. In the end the reproduced wavelets were very close to the ground truths regardless, at leas on the synthetically created datasets. These however turned out to be too far removed from the Kaggle dataset. The reasons for this are discussed in `train.ipynb`.

#### Dataset Creation
In order to create the datasets, I have altered the data creation code discussed in [this paper](https://arxiv.org/pdf/1904.08693.pdf) to allow for the use of Virgo data in the creation of synthetic datasets, as well as added some little convenience changes. The altered version should be available on my github shortly.

The data creation process goes as follows:

1. Real detector noise is downloaded. The choice to use real detector noise was made, because in numerous papers discussing machine learning applied to gravitational-wave signal detection, the result has always been, that deep learning networks perform much better on the colored Gaussian noise resulting from synthetic noise creation than on actual detector noise, even when transient "glitches" are injected into the noise.

2. Half of the samples are then injected with gravitational wave signals from black hole mergers chosen at random from all possible parameter sets (weight of the black holes, spin, etc.). The injections were limited to only black hole mergers as their duration will sensibly fit into the 2s window of data present in the Kaggle dataset. The assumption that only black hole mergers are used in the Kaggle dataset may be wrong. The simulated signals are injected and their signal to noise ratios(SNRs) are calculated using matched filtering. They are then scaled to produce SNRs in the chosen window, which was 3-20 in the synthetic datasets. This, however turned out not to work as expected, because in the actual testing datasets where the simulated masses were further away from the detectors were significantly easier for models trained on the Kaggle dataset to make predictions on than those with a closer distance, even though the reported SNR was the same in both cases.

In [4]:
def random_cut(x,y):
    tensor = x
    if random.random() > 0.65:
        maxVal=128
        dt = tf.random.uniform(shape=[],minval=2, maxval=maxVal, dtype=tf.int32)
        t0 = tf.random.uniform(shape=[],minval=1, maxval=dt, dtype=tf.int32)
        t1 = tf.random.uniform(shape=[],minval=0, maxval=t0, dtype=tf.int32)
        paddings =  [
            [0,0],
            [t0,dt-t0], #[t1, dt-t1] if you want to move the resulting tensor randomly in the output tensor
            [0,0]
        ]
        tensor = tf.pad(tensor[:,t0:t0+(4096-dt)], paddings=paddings)
#     tensor = tensor * [-1. if random.random() > 0.5 else 1.,
#                        -1. if random.random() > 0.5 else 1.,
#                        -1. if random.random() > 0.5 else 1.]
    # Necessary for TPU runtime
    tensor = tf.reshape(tensor,[Params["batch_size"], 4096, 3])
    # Necessary for GPU runtime
    tensor = tf.cast(tensor, tf.float32)
    return tensor, y

In [5]:
def load_dataset(files, cut=False):
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads = AUTO)
    
    def _parse_function(example_proto):
        keys_to_feature = {}
        keys_to_feature["TimeSeries"] = tf.io.FixedLenFeature([4096,3], tf.float32)
        keys_to_feature["GroundTruths"] = tf.io.FixedLenFeature([4096,3], tf.float32)
        
        parsed_features = tf.io.parse_single_example(example_proto, keys_to_feature)
        return parsed_features["TimeSeries"], parsed_features["GroundTruths"]
    
    dataset = dataset.map(_parse_function, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=10000, reshuffle_each_iteration=True)
    dataset = dataset.batch(Params["batch_size"])
    if cut:
        dataset = dataset.map(random_cut, num_parallel_calls=AUTO, deterministic=False)
    dataset = dataset.prefetch(AUTO)
    return dataset

### get_dataset_files()

This function divides the files into a training and validation dataset. The function can be given distances to choose from, because in testing it was shown, that even though the signal to noise ratio should be the same at all distances, the datasets with a higher distance between the detector and the event were much harder(distance in the filename is in Mpc). In the end the filter is meant to be trained using curriculum learning, where the dataset is made increasingly more difficult as training progresses.

In [6]:
def get_dataset_files(distances):
    train_files = []
    val_files = []
    for distance in distances:
        data_files = glob(f"../input/synthetic-tfrec/train_{distance}*.tfrec")
        train_files.extend(data_files[:-1])
        val_files.extend(data_files[-1:])
    return train_files, val_files

In [7]:
train_files,val_files = get_dataset_files([100,150,200,250,300,350])
train_ds = load_dataset(train_files)
val_ds = load_dataset(val_files)
train_files

['../input/synthetic-tfrec\\train_350_1_no_whiten_filter_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_350_1_no_whiten_filter_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_350_1_no_whiten_filter_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_350_1_no_whiten_filter_12500_03.tfrec',
 '../input/synthetic-tfrec\\train_350_2_no_whiten_filter_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_350_2_no_whiten_filter_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_350_2_no_whiten_filter_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_350_2_no_whiten_filter_12500_03.tfrec',
 '../input/synthetic-tfrec\\train_350_2_no_whiten_filter_12500_04.tfrec',
 '../input/synthetic-tfrec\\train_350_2_no_whiten_filter_12500_05.tfrec',
 '../input/synthetic-tfrec\\train_350_2_no_whiten_filter_12500_06.tfrec']

## Train
### Model

The first and current architecture used for the filter is a 3-channel version of the architecture proposed in [this paper](https://arxiv.org/abs/2105.03073). It is a encoder, decoder model, where the encoding is done using a simple 1D CNN and the decoding is done using a series of bidirectional LSTM layers. Unfortunately, so far I have not yet had the chance to experiment much with the architecture, because my initial datasets have proven insufficient for training (too easy, pre-processing is different to that of Kaggle dataset). I am currently making better datasets to be able to experiment more with training the filter.

The model is trained using root mean squared error between the predicted wave and the ground truth. This produces a model, that can reliably reproduce an approximation of the wave. The goal of the filter is, however to reduce the noise, increasing the SNR, so a different loss function will have to be created for that use case.

This custom initializer is meant as an initializer for a 1D convolutional layer to create a windowing effect, i.e. [x1,x2,x3,x4,x5,x5] => [[x1,x2,x3,x4],[x2,x3,x4,x5], ...].

In [8]:
def ownInitializer(shape, dtype=None):
    return tf.constant([
        [[1,0,0,0]],
        [[0,1,0,0]],
        [[0,0,1,0]],
        [[0,0,0,1]]
    ],dtype=dtype)

ownInitializer(2,tf.float32)

<tf.Tensor: shape=(4, 1, 4), dtype=float32, numpy=
array([[[1., 0., 0., 0.]],

       [[0., 1., 0., 0.]],

       [[0., 0., 1., 0.]],

       [[0., 0., 0., 1.]]], dtype=float32)>

Same principle but with 3 channels,which are added together. due to the loss of information this would produce, I decided to use a trainable and randomly initialized first layer. This was also done in large part due to time-constraints and will be re-visited.

In [9]:
model = tf.keras.Sequential([
    tf.keras.Input(shape = [4096,3]),
#         layers.Conv1D(kernel_size=12, filters=4,
#                        kernel_initializer=ownInitializer, input_shape=[4096,3], trainable=True, padding="same"),
    layers.Conv1D(kernel_size=4, filters=12, padding="same"),
#     layers.ZeroPadding1D(padding=[0,3]),
    layers.Reshape([4096,12,1]),
    layers.TimeDistributed(layers.Conv1D(kernel_size=3, filters=32, activation="tanh", padding="same")),
#     layers.TimeDistributed(layers.Conv1D(kernel_size=5, filters=32, activation="tanh", padding="same")),
    
    layers.TimeDistributed(layers.MaxPool1D()),
    layers.TimeDistributed(layers.Conv1D(kernel_size=3, filters=16, activation="tanh", padding="same")),
    layers.TimeDistributed(layers.Flatten()),
    
#     layers.Reshape([4093,32]),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    
# These layers need to be used when training on a TPU.
#     layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
#     layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
#     layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    
    
#     layers.Bidirectional(layers.LSTM(400)),
    
    
    layers.TimeDistributed(layers.Dense(3, dtype=tf.float32)),
])
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)

model.compile(
    optimizer=opt,
    metrics=["mean_squared_error", "cosine_similarity"],
    loss="mean_squared_error"
)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4096, 12)          156       
_________________________________________________________________
reshape (Reshape)            (None, 4096, 12, 1)       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 4096, 12, 32)      128       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4096, 6, 32)       0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4096, 6, 16)       1552      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4096, 96)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 4096, 256)         

In [10]:
train_steps = len(train_files) * 12500 // Params["batch_size"]
val_steps = len(val_files) * 12500 // Params["batch_size"]

In [11]:
from tensorflow.keras.callbacks import *
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2,
    patience=3, min_lr = 0.000001,
    verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=5
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
#     f"{MDL_PATH}/model_{Params['version']:03}.h5",
    "./model4.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weight_only=False,
    mode="auto",
    save_freq="epoch"
)


callbacks=[reduce_lr, early_stop, model_checkpoint]

### Training

In [ ]:
model.fit(train_ds,validation_data=val_ds, validation_steps=val_steps, steps_per_epoch=train_steps,
          epochs=60, callbacks=callbacks)

Epoch 1/60
1074/1074 [==============================] - 1201s 1s/step - loss: 0.0114 - mean_squared_error: 0.0114 - cosine_similarity: 0.0146 - val_loss: 0.0112 - val_mean_squared_error: 0.0112 - val_cosine_similarity: 0.0181
Epoch 2/60
1074/1074 [==============================] - 1162s 1s/step - loss: 0.0113 - mean_squared_error: 0.0113 - cosine_similarity: 0.0130 - val_loss: 0.0112 - val_mean_squared_error: 0.0112 - val_cosine_similarity: 0.0051
Epoch 3/60
1074/1074 [==============================] - 1166s 1s/step - loss: 0.0115 - mean_squared_error: 0.0115 - cosine_similarity: 0.0055 - val_loss: 0.0112 - val_mean_squared_error: 0.0112 - val_cosine_similarity: 0.0046
Epoch 4/60
1074/1074 [==============================] - 1177s 1s/step - loss: 0.0114 - mean_squared_error: 0.0114 - cosine_similarity: 0.0095 - val_loss: 0.0111 - val_mean_squared_error: 0.0111 - val_cosine_similarity: 0.0107

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/60
107

### Analysis

In [ ]:

predictions = model.predict(train_ds, steps=5)

In [ ]:
predictions.shape

In [ ]:
import matplotlib.pyplot as plt
for i in range(10,50):
    
    plt.plot(predictions[i])
    plt.show()

In [ ]:
for i in train_ds:
    plt.plot(i[1][7])
    break
    

In [ ]:
model.save("model5.h5")